# Formulário de perfil do corredor

Este notebook cria um formulário interativo para coletar dados de um novo corredor, estimar o nível atual (iniciante, intermediário ou avançado) e salvar as respostas em um arquivo JSON para uso posterior no sistema.


In [ ]:
from __future__ import annotations
import json
from datetime import datetime
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display


In [ ]:
# --- Prova alvo
prova_distancia = widgets.Dropdown(
    options=["5 km", "10 km", "21 km", "42 km", "Outra"],
    description="Distância",
    value="10 km",
    style={"description_width": "initial"},
)

data_prova = widgets.DatePicker(
    description="Data da prova",
    style={"description_width": "initial"},
)

terreno_prova = widgets.Dropdown(
    options=["Plana", "Com subidas", "Trilha", "Alta temperatura", "Altitude"],
    description="Terreno",
    value="Plana",
    style={"description_width": "initial"},
)

# --- Histórico e forma atual
melhor_tempo_min = widgets.BoundedIntText(
    value=28,
    min=10,
    max=120,
    step=1,
    description="Melhor tempo (min)",
    style={"description_width": "initial"},
)

melhor_tempo_seg = widgets.BoundedIntText(
    value=0,
    min=0,
    max=59,
    step=1,
    description="Segundos",
    style={"description_width": "initial"},
)

volume_semanal = widgets.BoundedIntText(
    value=30,
    min=0,
    max=200,
    step=1,
    description="Volume semanal (km)",
    style={"description_width": "initial"},
)

dias_treino = widgets.Dropdown(
    options=list(range(1, 8)),
    value=4,
    description="Dias/semana",
    style={"description_width": "initial"},
)

corrida_longa = widgets.BoundedIntText(
    value=12,
    min=0,
    max=100,
    step=1,
    description="Corrida longa (km)",
    style={"description_width": "initial"},
)

# --- Disponibilidade
tempo_livre_diario = widgets.BoundedIntText(
    value=60,
    min=0,
    max=300,
    step=5,
    description="Tempo livre diário (min)",
    style={"description_width": "initial"},
)

horas_semanais_total = widgets.Dropdown(
    options=["< 3 h", "3–5 h", "5–7 h", "> 7 h"],
    value="3–5 h",
    description="Horas/semana",
    style={"description_width": "initial"},
)

periodos_preferidos = widgets.SelectMultiple(
    options=["Manhã", "Tarde", "Noite"],
    value=("Manhã",),
    description="Períodos",
    style={"description_width": "initial"},
)

locais_treino = widgets.SelectMultiple(
    options=["Pista/esteira", "Rua/ciclovia", "Trilha", "Academia"],
    value=("Rua/ciclovia",),
    description="Locais",
    style={"description_width": "initial"},
)

# --- Objetivo
meta_principal = widgets.Dropdown(
    options=["Terminar prova", "Recorde pessoal", "Melhorar saúde", "Emagrecimento", "Outra"],
    value="Recorde pessoal",
    description="Meta principal",
    style={"description_width": "initial"},
)

meta_tempo_min = widgets.BoundedIntText(
    value=55,
    min=0,
    max=400,
    description="Meta de tempo (min)",
    style={"description_width": "initial"},
)

meta_tempo_seg = widgets.BoundedIntText(
    value=0,
    min=0,
    max=59,
    description="Segundos",
    style={"description_width": "initial"},
)

cross_training = widgets.SelectMultiple(
    options=["Musculação", "Bike", "Natação", "Nenhuma"],
    value=("Musculação",),
    description="Cross-training",
    style={"description_width": "initial"},
)

importancia_prova = widgets.SelectionSlider(
    options=[1, 2, 3],
    value=2,
    description="Importância (1–3)",
    continuous_update=False,
    style={"description_width": "initial"},
)

# --- Fatores pessoais e restrições
saude_lesoes = widgets.SelectMultiple(
    options=["Sem lesões", "Tendinite", "Fratura de estresse", "Dor lombar", "Outra"],
    value=("Sem lesões",),
    description="Saúde/lesões",
    style={"description_width": "initial"},
)

limitacoes_medicas = widgets.Dropdown(
    options=["Nenhuma", "Cardíaca", "Respiratória", "Metabólica", "Outra"],
    value="Nenhuma",
    description="Limitações",
    style={"description_width": "initial"},
)

peso = widgets.BoundedFloatText(
    value=70.0,
    min=0.0,
    max=300.0,
    step=0.1,
    description="Peso (kg)",
    style={"description_width": "initial"},
)

altura = widgets.BoundedFloatText(
    value=170.0,
    min=100.0,
    max=250.0,
    step=0.1,
    description="Altura (cm)",
    style={"description_width": "initial"},
)

horas_sono = widgets.BoundedFloatText(
    value=7.5,
    min=0.0,
    max=12.0,
    step=0.25,
    description="Sono (h/noite)",
    style={"description_width": "initial"},
)

estresse_diario = widgets.IntSlider(
    value=3,
    min=0,
    max=10,
    step=1,
    description="Estresse (0-10)",
    continuous_update=False,
    style={"description_width": "initial"},
)

# --- Suporte e motivação
grupo_treino = widgets.ToggleButtons(
    options=["Sim", "Não"],
    value="Sim",
    description="Grupo de treino",
    style={"description_width": "initial"},
)

motivacao = widgets.IntSlider(
    value=8,
    min=0,
    max=10,
    step=1,
    description="Motivação (0-10)",
    continuous_update=False,
    style={"description_width": "initial"},
)

compromissos = widgets.Dropdown(
    options=["Nenhum", "Alguns dias ocupados", "Agenda muito cheia"],
    value="Alguns dias ocupados",
    description="Compromissos",
    style={"description_width": "initial"},
)


In [ ]:
def estimar_nivel() -> str:
    score = 0

    # Volume semanal
    if volume_semanal.value > 60:
        score += 3
    elif volume_semanal.value >= 40:
        score += 2
    elif volume_semanal.value >= 20:
        score += 1

    # Melhor tempo em 5 km ou similar
    tempo_segundos = melhor_tempo_min.value * 60 + melhor_tempo_seg.value
    if tempo_segundos < 20 * 60:
        score += 3
    elif tempo_segundos <= 25 * 60:
        score += 2
    elif tempo_segundos <= 35 * 60:
        score += 1

    # Dias de treino
    if dias_treino.value >= 6:
        score += 3
    elif dias_treino.value >= 4:
        score += 2
    elif dias_treino.value >= 3:
        score += 1

    # Corrida longa recente
    if corrida_longa.value >= 30:
        score += 3
    elif corrida_longa.value >= 18:
        score += 2
    elif corrida_longa.value >= 12:
        score += 1

    if score >= 9:
        return "Avançado"
    if score >= 5:
        return "Intermediário"
    return "Iniciante"


def coletar_dados() -> dict:
    meta_tempo_segundos = meta_tempo_min.value * 60 + meta_tempo_seg.value
    tempo_segundos = melhor_tempo_min.value * 60 + melhor_tempo_seg.value
    dados = {
        "prova": {
            "distancia": prova_distancia.value,
            "data": data_prova.value.isoformat() if data_prova.value else None,
            "terreno": terreno_prova.value,
        },
        "historico_forma": {
            "melhor_tempo_segundos": tempo_segundos,
            "volume_semanal_km": volume_semanal.value,
            "dias_treino": dias_treino.value,
            "corrida_longa_km": corrida_longa.value,
        },
        "disponibilidade": {
            "tempo_livre_diario_min": tempo_livre_diario.value,
            "horas_semanais": horas_semanais_total.value,
            "periodos_preferidos": list(periodos_preferidos.value),
            "locais_treino": list(locais_treino.value),
        },
        "objetivo": {
            "meta_principal": meta_principal.value,
            "meta_tempo_segundos": meta_tempo_segundos,
            "cross_training": list(cross_training.value),
            "importancia_prova": importancia_prova.value,
        },
        "fatores_pessoais": {
            "saude_lesoes": list(saude_lesoes.value),
            "limitacoes_medicas": limitacoes_medicas.value,
            "peso_kg": peso.value,
            "altura_cm": altura.value,
            "horas_sono": horas_sono.value,
            "estresse": estresse_diario.value,
        },
        "suporte_motivacao": {
            "grupo_treino": grupo_treino.value,
            "motivacao": motivacao.value,
            "compromissos": compromissos.value,
        },
    }
    dados["nivel_estimado"] = estimar_nivel()
    return dados


def salvar_json(dados: dict) -> Path:
    nome_arquivo = f"perfil_corredor_form_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    destino = Path(nome_arquivo)
    destino.write_text(json.dumps(dados, ensure_ascii=False, indent=2))
    return destino


resultado_label = widgets.HTML(value="Preencha os campos e clique em **Salvar perfil**.")


def ao_clicar(_):
    dados = coletar_dados()
    arquivo = salvar_json(dados)
    resultado_label.value = (
        f"<b>Nível estimado:</b> {dados['nivel_estimado']}<br>"
        f"Perfil salvo em: <code>{arquivo.name}</code>"
    )

botao_salvar = widgets.Button(
    description="Salvar perfil",
    button_style="success",
    icon="save",
)
botao_salvar.on_click(ao_clicar)


In [ ]:
formulario = widgets.VBox(
    [
        widgets.HTML("<h3>Prova alvo</h3>"),
        widgets.HBox([prova_distancia, data_prova, terreno_prova]),
        widgets.HTML("<h3>Histórico e forma atual</h3>"),
        widgets.HBox([melhor_tempo_min, melhor_tempo_seg]),
        widgets.HBox([volume_semanal, dias_treino, corrida_longa]),
        widgets.HTML("<h3>Disponibilidade</h3>"),
        widgets.HBox([tempo_livre_diario, horas_semanais_total]),
        widgets.HBox([periodos_preferidos, locais_treino]),
        widgets.HTML("<h3>Objetivo</h3>"),
        widgets.HBox([meta_principal, meta_tempo_min, meta_tempo_seg]),
        widgets.HBox([cross_training, importancia_prova]),
        widgets.HTML("<h3>Fatores pessoais e restrições</h3>"),
        widgets.HBox([saude_lesoes, limitacoes_medicas]),
        widgets.HBox([peso, altura, horas_sono, estresse_diario]),
        widgets.HTML("<h3>Suporte e motivação</h3>"),
        widgets.HBox([grupo_treino, motivacao, compromissos]),
        widgets.HTML("<hr>"),
        botao_salvar,
        resultado_label,
    ]
)

formulario
